In [1]:
from gliner import GLiNER
import glob
import os
import pandas as pd

C:\Users\Arif Irfan Ibrahim\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = GLiNER.from_pretrained("urchade/gliner_multi")

Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 46995.00it/s]
C:\Users\Arif Irfan Ibrahim\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\transformers\convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [3]:
# get news data (parquet file)
news_folder = r'..\data'
parquet_files = glob.glob(os.path.join(news_folder, '*.parquet'))

In [4]:
# extract text from files
corpus_text = []
df = pd.read_parquet(r'..\data\malay_news.parquet')
corpus_text.extend(df['Title'].dropna().str.lower().tolist())
corpus_text.extend(df['Summary'].dropna().str.lower().tolist())

In [5]:
# save corpus to text file
corpus_file = r'..\data\malay_news_corpus.txt'

with open(corpus_file, 'w', encoding='utf-8') as f:
    for text in corpus_text:
        f.write(text + '\n')
print(f"Corpus size: {len(corpus_text)} sentences")

Corpus size: 3418 sentences


In [6]:
# set labels
labels = ["GPE","PERSON","ORG","FAC","MONEY","NORP","LOC","PRODUCT","EVENT",
          "PERCENT","WORK_OF_ART","TIME","ORDINAL","CARDINAL","QUANTITY","LAW"]

In [7]:
# process individual sentences
all_entities = []
current_position = 0

for sentence in corpus_text:
    if not sentence.strip():
        current_position += len(sentence) + 1
        continue
        
    # get predictions
    entities = model.predict_entities(sentence, labels)
    
    # adjust position
    for entity in entities:
        entity["start"] += current_position
        entity["end"] += current_position
        all_entities.append(entity)
    
    # update position
    current_position += len(sentence) + 1

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


C:\Users\Arif Irfan Ibrahim\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\gliner\data_processing\processor.py:296: UserWarning: Sentence of length 514 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
C:\Users\Arif Irfan Ibrahim\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\gliner\data_processing\processor.py:296: UserWarning: Sentence of length 407 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
C:\Users\Arif Irfan Ibrahim\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\gliner\data_processing\processor.py:296: UserWarning: Sentence of length 635 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been trunc

In [8]:
# duplicates
seen = set()
unique_entities = []
for entity in all_entities:
    identifier = (entity["text"], entity["label"], entity["start"], entity["end"])
    if identifier not in seen:
        seen.add(identifier)
        unique_entities.append(entity)

In [9]:
# convert to df
df = pd.DataFrame(unique_entities)
df.head()

,start,end,text,label,score
0,0,5,polis,ORG,0.904411
1,23,31,siasatan,EVENT,0.733970
2,32,39,program,ORG,0.677047
3,40,45,ehati,ORG,0.682098
4,46,52,wanita,PERSON,0.935187


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24295 entries, 0 to 24294
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   start   24295 non-null  int64  
 1   end     24295 non-null  int64  
 2   text    24295 non-null  object 
 3   label   24295 non-null  object 
 4   score   24295 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 949.2+ KB


In [11]:
# save as csv
df.to_csv(r'..\data\results_first_line.csv', index=False)